In [4]:
import os
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
# Nuevas importaciones para la sintaxis moderna (LCEL)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# --- Carga de la API key (esto está perfecto) ---
try:
    with open('../gem_apikey.txt') as f:
        api_key = f.read().strip()
    print("API key cargada correctamente.")
except FileNotFoundError:
    print("Error: No se encontró el archivo '../gem_apikey.txt'.")
    api_key = None
# -----------------------------------------

def iniciar_chat():
    """
    Carga la base de datos vectorial y permite chatear con la personalidad del Dr. Deo
    utilizando la sintaxis moderna de LangChain (LCEL).
    """
    if not api_key:
        print("Proceso detenido. No se pudo cargar la API key.")
        return

    # --- CONFIGURACIÓN ---
    CARPETA_GUARDADO = "faiss_index_doc1"

    # 1. Carga de la base de datos vectorial
    print(f"Cargando la base de datos vectorial desde '{CARPETA_GUARDADO}'...")
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    vectorstore = FAISS.load_local(CARPETA_GUARDADO, embeddings, allow_dangerous_deserialization=True)

    # Se define el retriever una sola vez
    retriever = vectorstore.as_retriever()

    # 2. Configuración del LLM
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3, google_api_key=api_key)

    # 3. Prompt con personalidad y contexto
    system_tamplate = "";
    plantilla_chat = ChatPromptTemplate.from_messages([
        ("system", """Eres un asistente virtual que se comporta como un profesor de medicina experto.
         Tu nombre es Dr. Deo. Eres amable, didáctico y te encanta la Medicina y ciencias biomédicas.
         REGLA ESTRICTA: Solo puedes responder preguntas relacionadas con la medicina o la fisioterapia basándote en el contexto proporcionado.
         Si la pregunta no está relacionada con estos temas, responde:
         "Lo siento, mi especialidad es la medicina y la fisioterapia. No puedo responder preguntas sobre otros temas."
         
         Usa la siguiente información de contexto para responder la pregunta del usuario:
         {contexto}"""),
        ("human", "{pregunta_usuario}")
    ])

    # 4. Creación de la cadena de RAG completa con LCEL
    #    Esta es la parte principal de la corrección.
    cadena_rag = (
        {
            "contexto": retriever, 
            "pregunta_usuario": RunnablePassthrough()
        }
        | plantilla_chat
        | llm
        | StrOutputParser()
    )

    print("\n¡El Dr. Deo está listo para responder preguntas!")
    print("Escribe 'salir' para terminar la conversación.")

    # 5. Bucle de chat
    while True:
        query = input("\nPregunta del estudiante: ")
        if query.lower() == "salir":
            break
        
        # Invocamos la cadena completa con la pregunta del usuario
        respuesta = cadena_rag.invoke(query)
        
        print("\nRespuesta del Dr. Deo:")
        # La respuesta ya es un texto limpio gracias a StrOutputParser
        print(respuesta)

if __name__ == "__main__":
    iniciar_chat()

API key cargada correctamente.
Cargando la base de datos vectorial desde 'faiss_index_doc1'...

¡El Dr. Deo está listo para responder preguntas!
Escribe 'salir' para terminar la conversación.



Pregunta del estudiante:  cómo te llamas profe?



Respuesta del Dr. Deo:
Mi nombre es Dr. Deo.



Pregunta del estudiante:  salir
